### Scrape Web pages for particular title

In [ ]:
!pip install llama-index

In [34]:
import requests
from bs4 import BeautifulSoup
import requests
import os
from llama_index.core import GPTVectorStoreIndex, SimpleDirectoryReader, ListIndex, Document
from google.colab import userdata
from dotenv import load_dotenv
openai_api_key=userdata.get('openai_api_key');
##load_dotenv()
##openai_api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_api_key

In [35]:
def scrape_job(job_title):
    url = f"https://www.jobly.fi/tyopaikat?search={job_title}" # Construct the search URL
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes

    soup = BeautifulSoup(response.content, "html.parser") # Parse the HTML content
    job_listings = soup.find_all("a", class_="recruiter-job-link") # Example selector: Replace with the actual class or id of the job listing container.

    job_links = [listing['href'] for listing in job_listings]
    top3_jobs=job_links[::2][:3]
    #print(top3_jobs)
    return top3_jobs


In [40]:
job_title="Data Scientist"
top3_jobs=scrape_job(job_title)
top3_jobs

['https://www.jobly.fi/tyopaikka/data-scientist-2229094',
 'https://www.jobly.fi/tyopaikka/data-scientist-2227309',
 'https://www.jobly.fi/tyopaikka/senior-scientist-background-farm-animal-genetics-2220729']

In [39]:
def job_description(scrape_result):
    dic = {}
    for i, link in enumerate(top3_jobs):
      response = requests.get(link)
      soup = BeautifulSoup(response.content, "html.parser")
      job_description_element = soup.find("div", class_="l-main")
      job_description = job_description_element.get_text(strip=True)
      job_description = job_description_element.get_text(strip=True)
      text = "Hae paikkaaTallenna työpaikka"
      first_occurrence = job_description.find(text)
      second_occurrence = job_description.find(text, first_occurrence + 1)
      extracted_text = job_description[first_occurrence+len(text):second_occurrence]

      dic[i] = extracted_text
    return dic

In [41]:
summary_3jobs = job_description(top3_jobs)

### If the text is in Finnish, then translate it to English

In [20]:
#!pip install python-dotenv

In [49]:
def semantic_search(query):
    insights = {}
    for item, summary in summary_3jobs.items():
      # Create a document object
      documents = [Document(text=summary)]
      index = GPTVectorStoreIndex.from_documents(documents)
      # Get a QueryEngine object from the index
      query_engine = index.as_query_engine()
      # Use the QueryEngine to query the index
      response = query_engine.query(query)
      insights[item] = response.response
    return insights


In [48]:
prompt = "Describe in 5 paragraphs: the job role/key responsibilities, the qualifications needed for the job and the key skills required for the job (both technical and soft skills), the domain and the seniority level. "
result=semantic_search(prompt)
result

{0: 'The job role entails working as a Data Scientist at Wärtsilä Voyage, focusing on the global Vessel Traffic Services (VTS) and Port Management Information Systems (PMIS) product lines. The key responsibilities include conceptualizing and specifying solution requirements in collaboration with business stakeholders, formalizing tasks, researching different approaches, designing and developing machine learning applications and algorithms, and building and optimizing machine learning pipelines. The role involves utilizing data from various sources to develop AI services that enhance safety, decarbonization, and operational efficiency in the maritime industry.\n\nTo qualify for the position, candidates should have a strong mathematical background in statistics and understanding of classical machine learning algorithms. Proficiency in software development using Python, familiarity with machine learning stack (NumPy, Pandas, Scikit-Learn, Pytorch), and web application stack (Fastapi/Flask

In [33]:
### get_insights(result)

### Specify the domain / field of industry

### Get all the seniority level skillsets required

### Summarize the task and skillset

### Highlight the top keywords / skillsets

### What are the other keywords that you might search for your similar positions

### Report the skills missing and visualize in Venn diagram, bubble diagram etc

### Suggest how to upskill based on current market